In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input/indoor-location-navigation/train'):
#    for filename in filenames:
#        print(dirname)
#        print(filename)
#        print(os.path.join(dirname, filename))
import sklearn as skl
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from dataclasses import dataclass


In [ ]:
myfile = open("/Users/neel2sharma/indoor-location-navigation/train/5a0546857ecc773753327266/F2/5dccf516c04f060006e6e3c9.txt")
txt = myfile.read()
print(txt)
myfile.close()

In [27]:
# copy from https://github.com/location-competition/indoor-location-competition-20/blob/master/io_f.py

@dataclass
class ReadData:
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray
    siteID: str
    timestamp_length: int

def read_data_file(data_filename):
    wifi = []
    ibeacon = []
    waypoint = []
    siteID = ''
    timestamp_length = 0

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        #print(line_data) added
        
        if 'SiteID:' in line_data:
            siteID = line_data.split('\t')[1].replace('SiteID:', '')

        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')
        
        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])
            continue
        
        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            timestamp_length += 1
            continue

        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), rssi]
            ibeacon.append(ibeacon_data)
            continue
    
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)
    #print(wifi)
    
    return ReadData(wifi, ibeacon, waypoint, siteID, timestamp_length)

sample_file = read_data_file("/Users/neel2sharma/indoor-location-navigation/train/5a0546857ecc773753327266/F2/5dccf516c04f060006e6e3c9.txt")

print('wifi shape:', sample_file.wifi.shape)
print('ibeacon shape:', sample_file.ibeacon.shape)
print('waypoint shape:', sample_file.waypoint.shape)
print('Timestamp length:', sample_file.timestamp_length)

wifi shape: (5258, 5)
ibeacon shape: (267, 3)
waypoint shape: (6, 3)
Timestamp length: 5258


In [ ]:
for dirname, _, filenames in os.walk('/Users/neel2sharma/indoor-location-navigation/train'):
    for filename in filenames:

        if (filename != ".DS_Store"):

            train_file = read_data_file(os.path.join(dirname, filename))
            waypoint_timestamps = train_file.waypoint[:,0]
            wifi_timestamps = [0] * len(waypoint_timestamps)

            i = 0
            while i < len(waypoint_timestamps):
                for x in train_file.wifi:
                    y = x[0].astype(np.float)
                    if abs(waypoint_timestamps[i] - y) < abs(waypoint_timestamps[i] - wifi_timestamps[i]):
                        wifi_timestamps[i] = y
                i += 1

            i = 0
            while i < len(wifi_timestamps):
                toAppend = []
                for x in train_file.wifi:
                    #check this logic
                    if wifi_timestamps[i].astype(float) == x[0].astype(float):
                        print('here')
                i += 1

In [19]:
#do some kind of selection of train files

#create train lists outside for use after for loops
train_x = []
train_yx = []
train_yy = []

for dirname, _, filenames in os.walk('/Users/neel2sharma/indoor-location-navigation/train'):
    for filename in filenames:
        if (filename == ".DS_Store"):
            continue
        train_file = read_data_file(os.path.join(dirname, filename))
        
        #if (train_file.siteID != '5c3c44b80379370013e0fd2b'):
        #    continue

        #obtain floorInt from file name
        floor = dirname[77:79]
        floorInt = 0
        if (floor == "F1"):
            floorInt = 1
        elif (floor == "F2"):
            floorInt = 2
        elif (floor == "F3"):
            floorInt = 3
        elif (floor == "F4"):
            floorInt = 4
            
        #obtain waypoint values
        waypoint_values_x = train_file.waypoint[:,1]
        waypoint_values_y = train_file.waypoint[:,2]
        waypoint_timestamps = train_file.waypoint[:,0]
        
        #Append x and y data for this file into train_yx and train_yy (lists rather than ndarrays for sklearn)
        i = 0
        while i < len(waypoint_values_x):
            train_yx.append(waypoint_values_x[i].astype(int))
            i += 1
            
        i = 0
        while i < len(waypoint_values_y):
            train_yy.append(waypoint_values_y[i].astype(int))
            i += 1
            
        #Select closest timestamps
        wifi_timestamps = [0] * len(waypoint_timestamps)
        i = 0
        while i < len(waypoint_timestamps):
            for x in train_file.wifi:
                y = x[0].astype(float)
                if abs(waypoint_timestamps[i] - y) < abs(waypoint_timestamps[i] - wifi_timestamps[i]):
                    wifi_timestamps[i] = y
            i += 1

        #loop through wifi_timestamps and subloop train_file.wifi
        #if timestamps match, append train_file.wifi[i][3] to toAppend
        #after each outer loop, append floorInt to toAppend, then append toAppend to train_x

        #create nparray of zeroes with len = 561 as toAppend. Instead of appending to toAppend, use i to update individual values
        #this way all arrays in train_x have same length
        i = 0
        while i < len(wifi_timestamps):
            toAppend = np.zeros(561)
            for x in train_file.wifi:
                #check this logic
                if wifi_timestamps[i].astype(float) == x[0].astype(float):
                    if x[3] == 'nan':
                        toAppend[i] = 0
                    else:
                        toAppend[i] = x[3].astype(int)
            toAppend[i] = floorInt #may be out of bounds, maybe do 562
            train_x.append(toAppend)
            #print(toAppend)
            i += 1
            
    #after the first loop, I have train_yx = [x1, x2, x3, x4, x5, x6], train_yy = [y1, y2, y3, y4, y5, y6]
    #train_x = [[rssi data1, floorInt1], [rssi data2, floorInt2], [rssi data3, floorInt3], [rssi data4, floorInt4], [rssi data5, floorInt5], [rssi data6, floorInt6]]
    #after each loop, train_yx and train_yy each have six more values and train_xx has six more lists
    #each list in train_x corresponds to the value in the same position in train_yx and train_yy

#need to figure out some way to make all lists in train_x the same size
list_train_x = np.array([np.array(xi) for xi in train_x])
#now fit MNB
MNBX = MultinomialNB()
MNBY = MultinomialNB()
        
MNBX.fit(list_train_x, train_yx)
MNBY.fit(list_train_x, train_yy)

#now format test data and run MNBX.predict(), MNBY.predict()

MultinomialNB()

In [20]:
#format train data and run predict

#somehow select timestamps
#create test_x in the same manner as train_x
#pass test_x into MNBX and MNBY

#problems
#  -need to choose timestamps appropriately (not sure how to do this, may need to look at train data file)
    # -files do not have the same timestamps or start times - maybe just use all rssi data? Not sure how big that would be. Or just choose random points?
    # -might be able to use all of the rssi data to a limit of (currently 561). Otherwise stick to the first 6 timestamps?
#  -may not have floorInt (check file name), if not present, remove from train data as well (no floor present in name)


test_x = []

for dirname, _, filenames in os.walk('/Users/neel2sharma/indoor-location-navigation/test'):
    for filename in filenames:
        if (filename == ".DS_Store"):
            continue
        train_file = read_data_file(os.path.join(dirname, filename))
        
       # if (train_file.siteID != '5c3c44b80379370013e0fd2b'):
        #    continue
        
        #problems here: wifi_timestamps doesn't exist - I don't know how many of the rssi rows to take
        #train_x looks like [[rssi11, rssi12, rssi13, rssi13, rssi14, rssi15, rssi16], [rssi21, rssi22, rssi23, rssi24, rssi25, rssi26], ...]
            #in the current filter, the longest such sublist is 561 points long
        i = 0
        while i < 6: #make a prediction for each timestamp
            toAppend = np.zeros(561) #I got 561 for the length of the longest rssi point
            
            if x[3] == 'nan':
                toAppend[i] = 0
            else:
                toAppend[i] = x[3].astype(int)
            test_x.append(toAppend)
            i += 1
    



#bnb and lr are giving lists of lists of the same value (ex [[97, 101], [97, 101], [97, 101],...])
#mnb is giving a 2D array where each diagonal line is repeats
    #honestly not sure what's going on - test data once prof responds, set up meeting if it fails
predicted_x = MNBX.predict(test_x)
predicted_y = MNBY.predict(test_x)

predicted_merge = []
i = 0
while (i < len(predicted_x)):
    toAppend = []
    toAppend.append(predicted_x[i])
    toAppend.append(predicted_y[i])
    predicted_merge.append(toAppend)
    i += 1
print(predicted_merge)

[[0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [267, 232], [278, 244], [0, 288], [291, 257], [280, 305], [268, 2], [26

In [ ]:
df = pd.DataFrame({"": , "": , "": })
df.to_csv("submission.csv", index=False)

In [29]:
sample_file = read_data_file("/Users/neel2sharma/indoor-location-navigation/train/5a0546857ecc773753327266/F2/5dccf516c04f060006e6e3c9.txt") # "/Users/neel2sharma/indoor-location-navigation/test/1a2cc4c5793fa089dd58ddd1.txt"
wifi_df = pd.DataFrame(sample_file.wifi)
wifi_df.columns = ['timestamp', 'ssid', 'bssid', 'rssi', 'last_seen_timestamp']
wifi_df = wifi_df.pivot(index='timestamp', columns=['ssid', 'bssid'])['rssi']
wifi_df.reset_index(drop=False, inplace=True)
wifi_df.style.set_caption('WiFi')

In [ ]:
for dirname, _, filenames in os.walk('/Users/neel2sharma/indoor-location-navigation/train'):
    find = False
    for filename in filenames:
        if (filename == ".DS_Store"):
            continue
        test = read_data_file(os.path.join(dirname, filename))
        if (test.siteID == '5a0546857ecc773753327266'):
            find = True
            file = os.path.join(dirname, filename)
            break
    if (find):
        break

print(os.path.join(dirname, filename))

In [14]:
from glob import glob
import shutil

y = []
for x in train_x:
    y.append(len(x))
print(max(y))

561


In [11]:
#if taking too long, adapt and run this
#will make directories for each siteID
#then adapt code in cell baove to run through only one directory

train_files = [file for file in glob('/Users/neel2sharma/indoor-location-navigation/train' + '/*.txt')]

for file in train_files:
    with open(file, encoding='utf-8') as f:
        for line in f:
            if 'SiteID:' in line:
                siteid = line.split('\t')[1].replace('SiteID:', '')
                site_dir = os.path.join('/Users/neel2sharma/indoor-location-navigation', siteid)
                if not os.path.isdir(site_dir):
                    os.mkdir(site_dir)
                break
    shutil.copy(file, site_dir)

print('/Users/neel2sharma/indoor-location-navigation/')